In [ ]:
# Imports

from random import shuffle
import csv
import re
import copy
import random
import itertools

In [ ]:
# Make input lists

startOrder = ['B', 'B', 'B', 'G', 'G', 'G', 'Y', 'Y', 'Y', 'R', 'R', 'R', 'D']

needOrder = startOrder
while (needOrder[0] == 'D' or needOrder[-1] == 'D'):
    shuffle(needOrder)

loveOrder = startOrder
while (loveOrder[0] == 'D' or loveOrder[-1] == 'D'):
    shuffle(loveOrder)

In [ ]:
# Read a CSV to a list

def readCSV(filename):
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        your_list = list(reader)
    
    deck = [[], [], [], [], [], []]
    for cards in your_list:
        for i, card in enumerate(cards):
            if (i < 4):
                if (i == 0):
                    thisCardColor = 'B'
                elif (i == 1):
                    thisCardColor = 'G'
                elif (i == 2):
                    thisCardColor = 'Y'
                elif (i == 3):
                    thisCardColor = 'R'

                if (card[:2] == "D "):
                    thisCardPoints = -1 * int(card[-1])
                    thisCardColor = 'D'
                    thisCardLeft = thisCardMiddle = thisCardRight = 'N'
                else:
                    thisCardLeft = card[0]
                    thisCardMiddle = card[1]
                    thisCardRight = card[2]

                    thisCardPoints = 1

                thisCard = [
                    thisCardPoints,
                    thisCardColor,
                    thisCardLeft,
                    thisCardMiddle,
                    thisCardRight
                ]
                deck[i].append(thisCard)
            else:
                deck[i].append(card)
    return deck
CSV = readCSV("GameData/game.csv")
print CSV

In [ ]:
# Print hand

def printHand(hand):
    score = ""
    color = ""
    bottom = ""
    
    for card in hand:
        left = ' ' if str(card[2]) == 'N' else str(card[2])
        middle = ' ' if str(card[3]) == 'N' else str(card[3])
        right = ' ' if str(card[4]) == 'N' else str(card[4])
        
        if (card[1] == 'R'):
            score += "\x1b[31m"
            color += "\x1b[31m"
            bottom += "\x1b[31m"
        elif (card[1] == 'G'):
            score += "\x1b[32m"
            color += "\x1b[32m"
            bottom += "\x1b[32m"
        elif (card[1] == 'B'):
            score += "\x1b[34m"
            color += "\x1b[34m"
            bottom += "\x1b[34m"
        elif (card[1] == 'Y'):
            score += "\x1b[33m"
            color += "\x1b[33m"
            bottom += "\x1b[33m"
        elif (card[1] == 'D'):
            score += "\x1b[35m"
            color += "\x1b[35m"
            bottom += "\x1b[35m"
        
        score += "|" + (' ' if card[0] > 0 and card[0] < 10 else '') + "" + str(card[0]) + " |  "
        color += "| " + str(card[1]) + " |  "
        bottom += "|" + left + "" + middle + "" + right + "|  "
    
    score += "\x1b[37m"
    color += "\x1b[37m"
    bottom += "\x1b[37m"
        
    print score
    print color
    print bottom

In [ ]:
# Play a card onto the table
# in - array of cards on the table
# in - card just played
# in - position that the card was played at
# out - updated cards on table

def playCard(playedCards, currCard, position):
    
    cardColor = currCard[1]
    cardLeft = currCard[2]
    cardMiddle = currCard[3]
    cardRight = currCard[4]
    playedCards.insert(position, currCard)
    
    oldPoints = []
    for i in range(len(playedCards)):
        oldPoints.append(playedCards[i][0])
    
    # Go through each card left to right
    for i, card in enumerate(playedCards):

        # See if this card's middle is the card just added
        if (card[3] == cardColor):
            if (cardColor == 'D'):
                playedCards[position][0] += 1
            else:
                card[0] += 1
        
        # See if this card is the played card's middle
        if (card[1] == cardMiddle and not i == position):
            if (cardMiddle == 'D'):
                card[0] += 1
            else:
                playedCards[position][0] += oldPoints[i]

        # The card to the left of the card just played
        if (i == position - 1):
            if (playedCards[i][4] == cardColor):
                if (cardColor == 'D'):
                    playedCards[position][0] += 1
                else:
                    card[0] += 1

        # The card to the right of the card just played
        if (i == position + 1):
            if (playedCards[i][2] == cardColor):
                if (cardColor == 'D'):
                    playedCards[position][0] += 1
                else:
                    card[0] += 1

        # The card just played
        if (i == position):
            if (i > 0):
                if (playedCards[i - 1][1] == cardLeft):
                    if (cardLeft == 'D'):
                        playedCards[position - 1][0] += 1
                    else:
                        card[0] += oldPoints[i - 1]
            if (i < len(playedCards) - 1):
                if (playedCards[i + 1][1] == cardRight):
                    if (cardRight == 'D'):
                        playedCards[position + 1][0] += 1
                    else:
                        card[0] += oldPoints[i + 1]

# Calculate the score of a hand
# in - array of cards on the table
# in - the needed color
# in - the loved color
# out - points from this hand

def countPoints(playedCards, need, love):
    if (not len(playedCards) == 4):
        raise ValueError('There are not 4 cards in this hand, there are ' + str(len(playedCards)))
        
    cardColors = []
    needCard = False
    loveCard = False
    darkCard = False
    points = 0
    
    # Put each card color into an array, make
    # sure the needed color was used and
    # add any points from cards
    for card in playedCards:
        thisColor = card[1]
        cardColors.append(thisColor)
        
        multiplier = 1
        if (thisColor == need):
            needCard = True
        if (thisColor == love):
            multiplier = 2
            loveCard = True
        if (thisColor == 'D'):
            darkCard = True
            
        points += card[0] * multiplier
        
    unique = len(list(set(cardColors))) == len(cardColors)
    
    # Return if the needed color was not used
    if (not needCard):
        return -1
    
    # Check if there are any bonus for this hand
    
    # Check if all the cards are the same
    if (cardColors[0] == cardColors[1] == cardColors[2] == cardColors[3]):
        points += 8
    # Check if 3 of the 4 cards are the same
    elif (cardColors[0] == cardColors[1] == cardColors[2] or cardColors[1] == cardColors[2] == cardColors[3]):
        if (loveCard):
            points += 8
        else:
            points += 4
    else:
        # Keep track if we will need to look at the rainbow bonus
        shouldCheckRainbow = True
        
        # Check if there is a pair
        for i in range(0, 3):
            if (cardColors[i] == cardColors[i + 1]):
                if (cardColors[i] == love):
                    points += 4
                else:
                    points += 2
                shouldCheckRainbow = False
        
        if (shouldCheckRainbow):
            if (not darkCard and unique):
                if (love == 'D'):
                    points += 3
                else:
                    points += 6
    
    return points

In [ ]:
# Tests

def playTest(thisTest, verify, testNum, debug=False):
    
    result = []
    for card in thisTest:
        cardCopy = copy.deepcopy(card)
        
        playCard(result, cardCopy[0], cardCopy[1])
        if (debug):
            printHand(result)
            print "\n"
        
    if (not result == verify):
        if (debug == False):
            unitTest(thisTest, verify, testNum, True)
        
        print "This test's output"
        print result
        print ""
        
        printHand(result)
        print "--------------------------------"
        print "should be"
        print "--------------------------------"
        printHand(verify)
        
        raise ValueError('Test ' + str(testNum) + ' Failed.\n')
        
def pointTest(thisHand, colorNeeded, colorLoved, verify, testNum, debug=False):
    
    result = countPoints(thisHand, colorNeeded, colorLoved)
    
    if (not result == verify):
        raise ValueError('Wanted ' + str(verify) + ' got ' + str(result))

In [ ]:
# Unit tests

# Tests for adding cards to the hand
test1 = [[[1, 'R', 'N', 'N', 'G'], 0],
         [[1, 'B', 'R', 'N', 'N'], 1],
         [[1, 'G', 'B', 'N', 'R'], 0],
         [[1, 'Y', 'N', 'N', 'B'], 2]]
test1Solution = [[2, 'G', 'B', 'N', 'R'], [1, 'R', 'N', 'N', 'G'], [3, 'Y', 'N', 'N', 'B'], [2, 'B', 'R', 'N', 'N']]
playTest(test1, test1Solution, 1)

test2 = [[[-3, 'D', 'N', 'N', 'N'], 0],
         [[1, 'R', 'D', 'N', 'N'], 1],
         [[1, 'Y', 'R', 'N', 'N'], 2],
         [[1, 'G', 'N', 'B', 'N'], 1]]
test2Solution = [[-2, 'D', 'N', 'N', 'N'], [1, 'G', 'N', 'B', 'N'], [1, 'R', 'D', 'N', 'N'], [2, 'Y', 'R', 'N', 'N']]
playTest(test2, test2Solution, 2)

test3 = [[[1, 'B', 'N', 'G', 'N'], 0],
         [[-3, 'D', 'N', 'N', 'N'], 0],
         [[1, 'G', 'N', 'R', 'N'], 1],
         [[1, 'B', 'B', 'N', 'N'], 3]]

test3Solution = [[-3, 'D', 'N', 'N', 'N'], [1, 'G', 'N', 'R', 'N'], [2, 'B', 'N', 'G', 'N'], [3, 'B', 'B', 'N', 'N']]
playTest(test3, test3Solution, 3)

test4 = [[[-2, 'D', 'N', 'N', 'N'], 0],
         [[-3, 'D', 'N', 'N', 'N'], 0],
         [[1, 'B', 'N', 'D', 'N'], 0],
         [[1, 'Y', 'N', 'N', 'D'], 1]]

test4Solution = [[1, 'B', 'N', 'D', 'N'], [1, 'Y', 'N', 'N', 'D'], [-1, 'D', 'N', 'N', 'N'], [-1, 'D', 'N', 'N', 'N']]
playTest(test4, test4Solution, 4)

test5 = [[[1, 'G', 'N', 'G', 'N'], 0],
         [[1, 'G', 'N', 'G', 'N'], 0],
         [[1, 'G', 'N', 'G', 'N'], 0],
         [[1, 'G', 'N', 'G', 'N'], 0]]

test5Solution = [[20, 'G', 'N', 'G', 'N'], [10, 'G', 'N', 'G', 'N'], [6, 'G', 'N', 'G', 'N'], [5, 'G', 'N', 'G', 'N']]
playTest(test5, test5Solution, 5)

test6 = [[[1, 'G', 'N', 'R', 'N'], 0],
         [[1, 'R', 'G', 'N', 'B'], 1],
         [[-2, 'D', 'N', 'N', 'N'], 0],
         [[1, 'B', 'N', 'R', 'N'], 3]]

test6Solution = [[-2, 'D', 'N', 'N', 'N'], [2, 'G', 'N', 'R', 'N'], [3, 'R', 'G', 'N', 'B'], [3, 'B', 'N', 'R', 'N']]
playTest(test6, test6Solution, 6)

# Tests for tracking end of hand points
test1 = [[1, 'Y', 'N', 'N', 'N'],
         [1, 'R', 'N', 'N', 'N'],
         [1, 'B', 'N', 'N', 'N'],
         [1, 'G', 'N', 'N', 'N']]
test1Solution = 11
test1Need = 'R'
test1Love = 'B'
pointTest(test1, test1Need, test1Love, test1Solution, 1)

test2 = [[2, 'G', 'N', 'N', 'N'],
         [1, 'R', 'N', 'N', 'N'],
         [3, 'Y', 'N', 'N', 'N'],
         [2, 'B', 'N', 'N', 'N']]
test2Solution = 15
test2Need = 'G'
test2Love = 'R'
pointTest(test2, test2Need, test2Love, test2Solution, 2)

test3 = [[2, 'Y', 'N', 'N', 'N'],
         [3, 'G', 'N', 'N', 'N'],
         [2, 'R', 'N', 'N', 'N'],
         [3, 'B', 'N', 'N', 'N']]
test3Solution = 19
test3Need = 'B'
test3Love = 'G'
pointTest(test3, test3Need, test3Love, test3Solution, 3)

test4 = [[2, 'Y', 'N', 'N', 'N'],
         [1, 'Y', 'N', 'N', 'N'],
         [1, 'R', 'N', 'N', 'N'],
         [-3, 'D', 'N', 'N', 'N']]
test4Solution = 4
test4Need = 'Y'
test4Love = 'R'
pointTest(test4, test4Need, test4Love, test4Solution, 4)

test5 = [[2, 'B', 'N', 'N', 'N'],
         [1, 'B', 'N', 'N', 'N'],
         [1, 'R', 'N', 'N', 'N'],
         [-1, 'D', 'N', 'N', 'N']]
test5Solution = 4
test5Need = 'B'
test5Love = 'D'
pointTest(test5, test5Need, test5Love, test5Solution, 5)

test6 = [[1, 'Y', 'N', 'N', 'N'],
         [4, 'Y', 'N', 'N', 'N'],
         [1, 'Y', 'N', 'N', 'N'],
         [4, 'G', 'N', 'N', 'N']]
test6Solution = 14
test6Need = 'Y'
test6Love = 'B'
pointTest(test6, test6Need, test6Love, test6Solution, 6)

test7 = [[-1, 'D', 'N', 'N', 'N'],
         [-3, 'D', 'N', 'N', 'N'],
         [-3, 'D', 'N', 'N', 'N'],
         [1, 'G', 'N', 'N', 'N']]
test7Solution = -2
test7Need = 'G'
test7Love = 'R'
pointTest(test7, test7Need, test7Love, test7Solution, 7)

In [ ]:
# Functions

def getTurnPermutations():
    baseString = [4, 1, 4, 2, 4, 3, 4, 4]
    currPerm = [0] * 8
    permList = []
    
    finishPerm = False
    while (not finishPerm):
        for i in range(8):
            pos = 7 - i
            if (currPerm[pos] / baseString[pos] == 1):
                if (pos == 0):
                    finishPerm = True
                    break
                currPerm[pos] = 0
                currPerm[pos - 1] += 1
        permList.append(''.join(str(e) for e in currPerm))
        currPerm[7] += 1
    
    return permList[:-1]

In [ ]:
# Monte Carlo

game = readCSV("GameData/game.csv")

maxFound = 0
maxHand = []
maxLove = ''
maxNeed = ''
maxHowPlayed = []

for repeat in range(100000):
    break
    
    howPlayed = []
    thisTurnMoves = [0]*4
    cardsTaken = [0]*4
    gameTotal = 0
    
    for playedHands in range(13):
        
        goodHand = False
        while (not goodHand):
            hand = []
            colors = ['B', 'G', 'Y', 'R']
            shuffle(colors)
            for playedCards in range(4):
                while (True):
                    pile = random.randint(0,3)
                    if (cardsTaken[pile] < 13):
                        break

                position = random.randint(0, playedCards)
                howPlayed.append([pile, position])
                thisCard = copy.deepcopy(game[pile][cardsTaken[pile]])
                playCard(hand, thisCard, position)

                cardsTaken[pile] += 1
                thisTurnMoves[pile] += 1

            print game[playedHands][4], game[playedHands][4]
            thisHandPoints = countPoints(hand, game[playedHands][4], game[playedHands][5])
            if (thisHandPoints <= 0):
                for i in range(len(thisTurnMoves)):
                    cardsTaken[i] -= thisTurnMoves[i]

        gameTotal += thisHandPoints
        
    if (gameTotal > maxFound):
        maxFound = gameTotal
        maxHand = hand
        maxNeed = colors[0]
        maxLove = colors[1]
        maxHowPlayed = howPlayed
            
print maxFound
print maxNeed
print maxLove
print maxHowPlayed
printHand(maxHand)

In [ ]:
# Method 2

game = readCSV("GameData/game3.csv")
gameCardsTaken = [0]*4
gameScore = 0

for hands in range(13):
    
    maxFound = 0
    maxIndex = 0
    permList = getTurnPermutations()

    for permCount, perm in enumerate(permList):
    
        hand = []
        cardsTaken = copy.deepcopy(gameCardsTaken)
        shouldTally = True

        for i in range(4):
            pile = int(perm[i * 2])

            if (cardsTaken[pile] > 12):
                shouldTally = False
                break

            cardToPlay = copy.deepcopy(game[pile][cardsTaken[pile]])
            cardsTaken[pile] += 1
            posToPlay = int(perm[i * 2 + 1])

            playCard(hand, cardToPlay, posToPlay)
        
        if (shouldTally):
            total = countPoints(hand, game[4][hands], game[5][hands])

            if (total > maxFound):
                maxFound = total
                maxIndex = permCount
    
    print maxFound
    gameScore += maxFound
    
    hand = []
    for i in range(4):
        pile = int(permList[maxIndex][i * 2])
        gameCardsTaken[pile] += 1
        
        cardToPlay = game[pile][cardsTaken[pile]]
        posToPlay = int(permList[maxIndex][i * 2 + 1])

        playCard(hand, cardToPlay, posToPlay)
        printHand(hand)

print "Final score: " + str(gameScore)

In [ ]:
a = [1, 2,3]
print a[:-1]